<a href="https://colab.research.google.com/github/Bhavit03/Project_Emotion_recog/blob/main/mobilenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

In [ ]:
# import tensorflow_hub as hub

In [ ]:
# model=hub.KerasLayer('https://tfhub.dev/google/object_detection/mobile_object_localizer_v1/1',trainable=True)

In [ ]:
input_shape=(224,224,1)

In [ ]:
model=tf.keras.applications.mobilenet_v2.MobileNetV2(
    input_shape=input_shape,
    alpha=1.4,
    include_top=True,
    weights=None,
    pooling='avg',
    classes=5,
    classifier_activation='softmax'
)

In [ ]:
model.summary()

Model: "mobilenetv2_1.40_224"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 1  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 112, 112, 48  432         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 112, 112, 48  192         ['Conv1[0][0]']                  
                                )                                              

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

In [ ]:
# def preprocess_image_input(input_images):
#   input_images = input_images.astype('float32')
#   output_ims = tf.image.resize(
#     input_images,
#     input_shape,
#     method=ResizeMethod.BILINEAR,
#     preserve_aspect_ratio=False,
#     antialias=False,
#     name=None
# )
#   return output_ims

In [ ]:
# INPUT PREPROCESSING FOR MOBILENETV2
def preprocessinp(input_images):
  output_ims =tf.keras.applications.mobilenet_v2.preprocess_input(input_images)

In [ ]:
# link="https://www.kaggle.com/datasets/jonathanoheix/face-expression-recognition-dataset"

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import os
from tensorflow.keras.preprocessing.image import load_img
import cv2

In [ ]:
!cd "/content/drive/MyDrive/emotion ai"

In [ ]:
keyfacialdf=pd.read_csv("/content/drive/MyDrive/emotion ai/icml_face_data.csv")

In [ ]:
keyfacialdf.head()

,emotion,pixels
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
3,2,20 17 19 21 25 38 42 42 46 54 56 62 63 66 82 1...
4,3,77 78 79 79 78 75 60 55 47 48 58 73 77 79 57 5...


In [ ]:
type(keyfacialdf[' pixels'][0])

str

In [ ]:
def string2array(x):
  string=x.split(' ')
  return np.array(string).reshape(48, 48, 1).astype('float32')

In [ ]:
keyfacialdf.shape

(24568, 2)

In [ ]:
def resize(x):
  img=x.reshape(48,48)
  return cv2.resize(img,dsize=(224,224),interpolation=cv2.INTER_CUBIC)

In [ ]:
keyfacialdf[' pixels']=keyfacialdf[' pixels'].apply(lambda x:string2array(x))

In [ ]:
keyfacialdf[' pixels'][0].shape

(48, 48, 1)

In [ ]:
keyfacialdf[' pixels']=keyfacialdf[' pixels'].apply(lambda x:resize(x))

In [ ]:
keyfacialdf[' pixels'][0].shape

(224, 224)

In [ ]:
label_to_text={0:'anger',1:'disgust',2:'sad',3:'happiness',4:'surprise'}

In [ ]:
# emotions=[0,1,2,3,4]

# for i in emotions:
#   data=keyfacialdf[keyfacialdf['emotion']==i][:1]
#   img=data[' pixels'].item()
#   # img=img.reshape(128,128)
#   plt.figure()
#   plt.title(label_to_text[i])
#   plt.imshow(img,cmap='gray')

In [ ]:
from keras.utils import to_categorical
X=keyfacialdf[' pixels']
y=to_categorical(keyfacialdf['emotion'])

In [ ]:
X.shape

(24568,)

In [ ]:
X=np.stack(X,axis=0)
X=X.reshape(24568,224,224,1)

print(X.shape,y.shape)

(24568, 224, 224, 1) (24568, 5)


In [ ]:
X_train,X_test,y_train,y_test= X[:19650,:,:,:], X[19650:,:,:,:], y[:19650,:], y[19650:,:]

In [ ]:
del X,y, keyfacialdf

In [ ]:
# from sklearn.model_selection import train_test_split
# X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)
# X_val,X_test,y_val,y_test=train_test_split(X_test,y_test,test_size=0.5,shuffle=True)

In [ ]:
X_train=X_train/255.0
X_test=X_test/255.0
# X_val=X_val/255

NameError: ignored

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
traindatagen=ImageDataGenerator(
  rotation_range=15,
  width_shift_range=0.1,
  height_shift_range=0.1,
  shear_range=0.1,
  zoom_range=0.1,
  horizontal_flip=True,
  fill_mode="nearest"
)

In [ ]:
# checkpointer = ModelCheckpoint(filepath = "FacialKeyPoints_weights.hdf5", verbose = 1, save_best_only = True)
history=model.fit((X_train, y_train),epochs=10,batch_size=32)